In [1]:
from llama_index.readers import SimpleDirectoryReader, WikipediaReader, SimpleWebPageReader
from llama_index import download_loader
from llama_index.schema import MetadataMode
import dotenv
dotenv.load_dotenv('../.env')
from projectgurukul.readers import CSVReader,RamayanaCSVReader
import pandas as pd
import re

# Gita

In [3]:
def preprocess(row):
    row['chapter'], row['verse'] = row.verse_number.split(', ')
    return row


reader = CSVReader(text_columns=['verse','verse_in_sanskrit','translation_in_english','meaning_in_english'], metadata_columns=['chapter'], preprocess = preprocess)

documents = reader.load_data('../data/gita/data/bhagavad_gita.csv', extra_info={'souce': 'Bhagavad Gita'})

In [4]:
len(documents)

701

In [5]:
documents[0].metadata_template = "> {key}: {value}"
print(documents[0].get_content(metadata_mode = MetadataMode.ALL))

> chapter: Chapter 1
> souce: Bhagavad Gita

Verse 1

धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः |मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय ||1||

Dhritarashtra said: O Sanjay, after gathering on the holy field of Kurukshetra, and desiring to fight, what did my sons and the sons of Pandu do?

The two armies had gathered on the battlefield of Kurukshetra, well prepared to fight a war that was inevitable. Still, in this verse, King Dhritarashtra asked Sanjay, what his sons and his brother Pandu’s sons were doing on the battlefield? It was apparent that they would fight, then why did he ask such a question?The blind King Dhritarashtra’s fondness for his own sons had clouded his spiritual wisdom and deviated him from the path of virtue. He had usurped the kingdom of Hastinapur from the rightful heirs; the Pandavas, sons of his brother Pandu. Feeling guilty of the injustice he had done towards his nephews, his conscience worried him about the outcome of this battle.The words dhar

In [4]:
documents[1].get_metadata_str()

'chapter: Chapter 1\nsouce: Bhagavad Gita'

In [21]:
wikireader = WikipediaReader()
wikidocs = wikireader.load_data(['ayodhya ram temple'])


In [23]:
print( wikidocs[0].get_content()[:1500])

The Ram Mandir is a Hindu temple that is under construction in Ayodhya, Uttar Pradesh, India. It is located at the site of Ram Janmabhoomi, the hypothesized birthplace of Rama, a principal deity of Hinduism. The site is the former location of the Babri Masjid which was built after the demolition an existing non-Islamic structure. The worship of Hindu god Ram and Sita at the disputed site started when their idols were installed in 1949. In 2019, the Supreme Court of India delivered the verdict to give the disputed land to Hindus for a temple of Ram, while Muslims would be given land elsewhere to construct a mosque. The court referenced a report from the Archaeological Survey of India (ASI) as evidence suggesting the presence of a structure beneath the demolished Babri Masjid, that was found to be non-Islamic.The bhumi pujan (transl. ground breaking ceremony) for the commencement of the construction of Ram Mandir was performed on 5 August 2020, by Prime Minister Narendra Modi. The temple

# Ramayana

In [2]:
df = pd.read_csv("../data/ramayana/data/yuddhakanda.csv").dropna(how = 'all')
df.tail()

,content,explanation
5204,एवमेतत्पुरावृत्तमाख्यानंभद्रमस्तुवः ।।6.131.12...,This way Ramayana is the ancient narration. Re...
5205,देवाश्चसर्वेतुष्यन्तिग्रहणाच्छ्रवणात्तथा ।।6.1...,"By grasping and listening to Ramayana, all god..."
5206,भक्त्यारामस्ययेचेमांसंसितामृषिणाकृताम् ।।6.131...,Residence in heaven is assured for those who t...
5207,कुटुम्बवृद्धिंधनधान्यवृद्धिंस्त्रियश्चमुख्यास्...,Listening to this great auspicious epic will r...
5208,आयुष्यमारोग्यकरंयशस्यंसौभ्रातृकंबुद्धिकरंशुभं ...,If the narrative of Ramayana is regularly hear...


In [3]:
docs = RamayanaCSVReader().load_data("../data/ramayana/data/yuddhakanda.csv")

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131]


In [4]:
print(docs[-2].metadata)

{'sarga': 130, 'summary': ' Arrangements at Ayodhya to receive Sri Rama. Bharata goes to receive him.'}


In [6]:
print(docs[-2].get_content())

Hearing the delightful news, Bharata of truthful prowess, the destroyer of enemies, was very happy and ordered Shatrughna. ।।6.130.1।।

"Let all gods in the temples of the capital be worshiped by people purifying themselves, duly adorned with fragrance and garlands." ।।6.130.2।।

"Let the bards well versed in singing praises, charioteers, all courtesans, in groups, artists who can play musical instruments, similarly ministers, army men, traders, service providers, Kshatriyas, brahmins so also top people, troops proceed to see the moon like face of Rama." ।।6.130.4।।

Shatrughna, a destroyer of enemies, hearing the words of Bharata, ordered for many divisions of labour and the courageous ones to come. ।।6.130.5।।

From here (Nandigrama to Ayodhya) low lying areas are to be made even and uneven areas to be levelled. ।।6.130.6।।

Let the ground be sprinkled with cold water and then parched grain and flowers to be sprinkled. ।।6.130.7।।

Let the streets of the foremost of the best cities (